In [1]:
import pickle
import numpy as np

In [2]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    def forward(self, input):
        pass
    def backward(self, output_grad, learning_rate):
        pass

In [19]:
class Dense(Layer):
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias

    def forward(self, input):
        self.input = input
        print("forward: ",self.weights.shape,self.input.shape)
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        print("backward: ",output_gradient.shape,self.input.shape)
        weights_gradient = np.dot(np.expand_dims(output_gradient, axis=-1), np.expand_dims(self.input, axis=-1).T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient

In [4]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

In [5]:
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)

In [6]:
class Relu(Activation):
    def __init__(self):
        def Relu(x):
            return np.maximum(0, x)

        def Relu_prime(x):
            
            return np.where(x<=0,0,1)

        super().__init__(Relu, Relu_prime)

In [7]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)

def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

In [21]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

def train(network, loss, loss_prime, x_train, y_train, epochs = 5, learning_rate = 0.01, verbose = True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x)

            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)
        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")

In [9]:
def dataloader():
    
    with open('param.pkl','rb') as f:
        data = pickle.load(f)
    inputs = data["inputs"]
    w1 = data["w1"]
    w2 = data["w2"]
    w3 = data["w3"]
    b1 = data["b1"]
    b2 = data["b2"] 
    b3 = data["b3"]
    targets = data["targets"] 
    params = [
        (w1, b1),
        (w2, b2),
        (w3, b3)
    ]
    trainset = (inputs, targets)

    return trainset, params


In [22]:
trainset, params = dataloader()
network = [
    Dense(params[0][0],params[0][1]),
    Sigmoid(),
    Dense(params[1][0],params[1][1]),
    Sigmoid(),
    Dense(params[2][0],params[2][1]),
    Sigmoid()
]

# train
train(network, mse, mse_prime, trainset[0], trainset[1], epochs=5, learning_rate=0.01)


forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forw

forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
forward:  (1, 10) (10,)
backward:  (1,) (10,)
backward:  (10,) (10,)
backward:  (10,) (2,)
forward:  (10, 2) (2,)
forward:  (10, 10) (10,)
fo

In [ ]:
params[2][1]